In [ ]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=8f9ed798afaccd17a0f9cafeb7307c857637ae60f5d4b3d7d2203d46c160db05
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import folium
import wget
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)


In [ ]:
site_map

In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [ ]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [ ]:
# Mark the success/failed launches for each site on the map

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through each launch site and add markers for successful and failed launches
for index, record in spacex_df.iterrows():
    # Create a folium.Marker object for each launch
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='green' if record['class'] == 1 else 'red', icon_color='white', icon='info-sign'),
        popup=record['Launch Site']
    )
    # Add the marker to the MarkerCluster object
    marker_cluster.add_child(marker)

# Add the MarkerCluster object to the map
site_map.add_child(marker_cluster)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# prompt: # find coordinate of the closet coastline
# # e.g.,: Lat: 28.56367  Lon: -80.57163
# # distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Function to find the closest coastline point to a given coordinate
def find_closest_coastline(launch_site_lat, launch_site_lon):
    # This is a placeholder.  In a real application, you'd load coastline data
    # and perform a spatial query to find the nearest coastline point.
    # Replace this with your actual coastline data processing and nearest neighbor search.

    # Example (replace with your actual coastline data and logic):
    coastline_data = {
        'point1': {'lat': 28.56, 'lon': -80.57},  # Example coastline point
        'point2': {'lat': 28.60, 'lon': -80.60}   # Example coastline point
    }

    min_distance = float('inf')
    closest_coastline = None
    for point_name, point in coastline_data.items():
        distance = calculate_distance(launch_site_lat, launch_site_lon, point['lat'], point['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_coastline = point

    return closest_coastline, min_distance


# Example usage (replace with your actual coordinates):
launch_site_lat = 28.56367
launch_site_lon = -80.57163

closest_point, distance = find_closest_coastline(launch_site_lat, launch_site_lon)

print(f"Closest coastline point: Latitude={closest_point['lat']}, Longitude={closest_point['lon']}")
print(f"Distance to coastline: {distance} km")

Closest coastline point: Latitude=28.56, Longitude=-80.57
Distance to coastline: 0.43817311396118086 km


In [ ]:
# prompt: # Create and add a folium.Marker on your selected closest coastline point on the map
# # Display the distance between coastline point and launch site using the icon property
# # for example
# # distance_marker = folium.Marker(
# #    coordinate,
# #    icon=DivIcon(
# #        icon_size=(20,20),
# #        icon_anchor=(0,0),
# #        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
# #        )
# #    )

# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property

coordinate = [closest_point['lat'], closest_point['lon']]
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
site_map.add_child(distance_marker)

site_map

In [ ]:
coordinates = [[launch_site_lat, launch_site_lon], [closest_point['lat'], closest_point['lon']]]
coordinates = [[launch_site_lat, launch_site_lon], [closest_point['lat'], closest_point['lon']]]

lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)


In [ ]:
# prompt: # Create a marker with distance to a closest city, railway, highway, etc.
# # Draw a line between the marker to the launch site

# Assuming you have already installed the necessary libraries and have the site_map object
# from the previous code.

# Function to find the closest landmark (replace with your actual data and logic)
def find_closest_landmark(launch_site_lat, launch_site_lon, landmark_type):
    # Replace this with your actual landmark data and nearest neighbor search
    landmark_data = {
        'city': {'lat': 28.60, 'lon': -80.60, 'distance': 5}, #example
        'railway': {'lat': 28.58, 'lon': -80.58, 'distance': 2}, #example
        'highway': {'lat': 28.57, 'lon': -80.59, 'distance': 3}, #example
    }
    if landmark_type not in landmark_data:
      return None, None

    closest_landmark = landmark_data[landmark_type]
    return closest_landmark, closest_landmark['distance']


# Example usage:
launch_site_lat = 28.56367
launch_site_lon = -80.57163

landmark_types = ['city', 'railway', 'highway'] # Add your landmark types
for landmark_type in landmark_types:
    closest_landmark, distance = find_closest_landmark(launch_site_lat, launch_site_lon, landmark_type)

    if closest_landmark:
        coordinate = [closest_landmark['lat'], closest_landmark['lon']]
        landmark_marker = folium.Marker(
            coordinate,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f"{landmark_type}: {distance} km",
            )
        )
        site_map.add_child(landmark_marker)

        # Draw a line from the launch site to the landmark
        coordinates = [[launch_site_lat, launch_site_lon], [closest_landmark['lat'], closest_landmark['lon']]]
        lines = folium.PolyLine(locations=coordinates, weight=1, color='blue') # Customize color
        site_map.add_child(lines)
    else:
      print(f"No data found for {landmark_type}")

site_map

In [ ]:
# prompt: After you plot distance lines to the proximities, you can answer the following questions easily:
# - Are launch sites in close proximity to railways?
# - Are launch sites in close proximity to highways?
# - Are launch sites in close proximity to coastline?
# - Do launch sites keep certain distance away from cities?

# Assuming you have already installed the necessary libraries and have the site_map object
# from the previous code.

# Function to find the closest landmark (replace with your actual data and logic)
def find_closest_landmark(launch_site_lat, launch_site_lon, landmark_type):
    # Replace this with your actual landmark data and nearest neighbor search
    landmark_data = {
        'city': {'lat': 28.60, 'lon': -80.60, 'distance': 5}, #example
        'railway': {'lat': 28.58, 'lon': -80.58, 'distance': 2}, #example
        'highway': {'lat': 28.57, 'lon': -80.59, 'distance': 3}, #example
    }
    if landmark_type not in landmark_data:
      return None, None

    closest_landmark = landmark_data[landmark_type]
    return closest_landmark, closest_landmark['distance']


# Example usage:
launch_site_lat = 28.56367
launch_site_lon = -80.57163

landmark_types = ['city', 'railway', 'highway'] # Add your landmark types
for landmark_type in landmark_types:
    closest_landmark, distance = find_closest_landmark(launch_site_lat, launch_site_lon, landmark_type)

    if closest_landmark:
        coordinate = [closest_landmark['lat'], closest_landmark['lon']]
        landmark_marker = folium.Marker(
            coordinate,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f"{landmark_type}: {distance} km",
            )
        )
        site_map.add_child(landmark_marker)

        # Draw a line from the launch site to the landmark
        coordinates = [[launch_site_lat, launch_site_lon], [closest_landmark['lat'], closest_landmark['lon']]]
        lines = folium.PolyLine(locations=coordinates, weight=1, color='blue') # Customize color
        site_map.add_child(lines)
    else:
      print(f"No data found for {landmark_type}")

site_map